In [1]:
# インポート
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn import metrics

In [2]:
# データセットインストール
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

Using TensorFlow backend.
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py

In [3]:
print(X_train.shape) # (60000, 28, 28)
print(X_test.shape) # (10000, 28, 28)
print(X_train[0].dtype) # uint8

(60000, 28, 28)
(10000, 28, 28)
uint8


In [4]:
X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)
X_train /= 255
X_test /= 255
print(X_train.max()) # 1.0
print(X_train.min()) # 0.0
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train_one_hot = enc.fit_transform(y_train[:, np.newaxis])
y_test_one_hot = enc.transform(y_test[:, np.newaxis])
print(y_train.shape) # (60000,)
print(y_train_one_hot.shape) # (60000, 10)
print(y_train_one_hot.dtype) # float64

1.0
0.0
(60000,)
(60000, 10)
float64


In [5]:
#NCHW(n_samples, n_channels, height, width):採用

X_train_0, X_val_0, y_train_0, y_val_0 = train_test_split(X_train, y_train_one_hot, test_size=0.99)
# print(X_train_1.shape) # (600, 28,28)
# print(X_val.shape) # (59400, 28,28)
#Xにaxis1にchanelを追加する

X_train_0 = X_train_0[:, np.newaxis, :, :]
X_val_0 = X_val_0[:, np.newaxis, :, :]

print(X_train_0.shape) # (600, 28,28,1)
print(X_val_0.shape) # (59400, 28,28,1)
print(y_train_0.shape) # (600, 10)
print(y_val_0.shape) # (59400, 10)

(600, 1, 28, 28)
(59400, 1, 28, 28)
(600, 10)
(59400, 10)


In [6]:
#NHWC(n_samples, height, width, n_channels)：不採用

X_train_1, X_val, y_train_1, y_val = train_test_split(X_train, y_train_one_hot, test_size=0.99)
# print(X_train_1.shape) # (600, 28,28)
# print(X_val.shape) # (59400, 28,28)
#Xにchanelを追加するためreshape

X_train_1 = X_train_1.reshape(X_train_1.shape[0],X_train_1.shape[1],X_train_1.shape[2],1)
X_val = X_val.reshape(X_val.shape[0],X_val.shape[1],X_val.shape[2],1)

print(X_train_1.shape) # (600, 28,28,1)
print(X_val.shape) # (59400, 28,28,1)
print(y_train_1.shape) # (600, 10)
print(y_val.shape) # (59400, 10)

(600, 28, 28, 1)
(59400, 28, 28, 1)
(600, 10)
(59400, 10)


# 【問題2】2次元畳み込み後の出力サイズ

$$N_{h,out} =  \frac{N_{h,in}+2P_{h}-F_{h}}{S_{h}} + 1$$
$$N_{w,out} =  \frac{N_{w,in}+2P_{w}-F_{w}}{S_{w}} + 1$$

In [7]:
def Nh_out(Nh_in,Ph=0,Fh=3,Sh=1):
    Nh_out = (Nh_in + 2*Ph - Fh)/Sh + 1
    return int(Nh_out)

In [8]:
print(Nh_out(28))

26


In [9]:
def Nw_out(Nw_in,Pw=0,Fw=3,Sw=1):
    Nw_out = (Nw_in + 2*Pw - Fw)/Sw + 1
    return int(Nw_out)

In [10]:
print(Nw_out(28))

26


# 【問題3】最大プーリング層の作成

In [11]:
#入力はNHWC(n_samples, height, width, n_channels)
def MaxPool2D(A):
    sample = A.shape[0]
    chanel = A.shape[1]
    height = A.shape[2]
    width = A.shape[3]
    Z_index = np.zeros([sample,chanel,height,width])
    Z = np.zeros([sample,chanel,int(height/2),int(width/2)])
#     print(Z.shape)
    
    for h in range(sample):#サンプルを設定
        for k in range(chanel):#チャネルを設定
            for i in range(0, int(height/2), 1):#高さを設定
                for j in range(0, int(width/2), 1):#幅を設定
                        Z0 = A[h,k,i*2:i*2 + 2,j*2:j*2 + 2]
                        Z[h,k,i,j] = np.nanmax(Z0)
                        a = Z0/np.nanmax(Z0)#最大値が１の行列
                        Z_index[h,k,i*2:i*2 + 2,j*2:j*2 + 2] += (np.where(a==1,1,0))

    return Z, Z_index

In [12]:
test = np.array([1,3,2,9,7,4,1,5,8,5,2,3,4,2,1,4]).reshape(1,1,4,4)

In [13]:
test

array([[[[1, 3, 2, 9],
         [7, 4, 1, 5],
         [8, 5, 2, 3],
         [4, 2, 1, 4]]]])

In [14]:
MaxPool2D(test)

(array([[[[7., 9.],
          [8., 4.]]]]), array([[[[0., 0., 0., 1.],
          [1., 0., 0., 0.],
          [1., 0., 0., 0.],
          [0., 0., 0., 1.]]]]))

In [15]:
Z,Z_index = MaxPool2D(test)
print(Z_index)

[[[[0. 0. 0. 1.]
   [1. 0. 0. 0.]
   [1. 0. 0. 0.]
   [0. 0. 0. 1.]]]]


# 【問題1】2次元畳み込み層の作成

1次元畳み込み層のクラスConv1dを発展させ、2次元畳み込み層のクラスConv2dを作成してください。<br>
フォワードプロパゲーションの数式は以下のようになります。<br>

$$a_{i,j,m} = \sum_{k=0}^{K-1}\sum_{s=0}^{F_{h}-1}\sum_{t=0}^{F_{w}-1}x_{(i+s),(j+t),k}w_{s,t,k,m}+b_{m}$$

In [16]:
#テスト用のフィルター(Nout=2,Nin=1,h=2,w=2)
W0 = np.array([1,1,1,1,2,2,2,2]).reshape(2,1,2,2)
print(W0)

[[[[1 1]
   [1 1]]]


 [[[2 2]
   [2 2]]]]


In [17]:
#テスト用のバイアス(Nout=2)
B0 = np.array([1,2]).reshape(2)
print(B0)

[1 2]


In [18]:
#フォワードプロパゲーションのテスト
def forward(X,W,B):
    sample = X.shape[0]
    chanel = X.shape[1]
    height = X.shape[2]
    width = X.shape[3]
    
    c_out = W.shape[0]
    c_in = W.shape[1]
    w_height = W.shape[2]
    w_width = W.shape[3]
    
    h_out = Nh_out(Nh_in=height,Ph=0,Fh=2,Sh=1)
    w_out = Nw_out(Nw_in=width,Pw=0,Fw=2,Sw=1)
    
    A0_list = np.zeros([sample,c_out,c_in,h_out,w_out])
#     A_list = []
#     print(h_out)
#     print(w_out)
    
    # X*Wの計算
    for h in range(sample):#サンプルを設定
        for k in range(chanel):#チャネルを設定
            for i in range(0, h_out, 1):#高さを設定
                for j in range(0, w_out, 1):#幅を設定
                    A0 = X[h,k,i:i + w_height,j:j + w_width]#Xの中の要素を抽出
#                     print(A0)
                    for l in range(c_out):#フィルターの出力チャネルを設定
                        for m in range(c_in):#フィルターの入力チャネルを設定
                            A0_list[h,l,m,i,j] = (A0*W[l,m]).sum()

    A0_list = A0_list.sum(axis=2)#入力チャネル方向にSUM
    # +Bの計算
    for n in range(sample):#サンプルを設定
        for o in range(c_out):#フィルターの出力チャネルを設定
            print(A0_list[n,o,:,:].shape)
            print(B[o])
            A0_list[n,o,:,:] += B[o]
    
    return A0_list# A0_list([サンプル数,チャネル,高さ,幅])

In [20]:
np.array([1,1,1,1])[3]

1

In [21]:
list_a = forward(test,W0,B0)

(3, 3)
1
(3, 3)
2


In [22]:
list_a

array([[[[16., 11., 18.],
         [25., 13., 12.],
         [20., 11., 11.]],

        [[32., 22., 36.],
         [50., 26., 24.],
         [40., 22., 22.]]]])

In [23]:
#MAX POOL用テストデータ
test_pool = np.array(range(25)).reshape(1,1,5,5)
print(test_pool)

[[[[ 0  1  2  3  4]
   [ 5  6  7  8  9]
   [10 11 12 13 14]
   [15 16 17 18 19]
   [20 21 22 23 24]]]]


In [24]:
list_b = forward(test_pool,W0,B0)

(4, 4)
1
(4, 4)
2


In [25]:
list_b

array([[[[ 13.,  17.,  21.,  25.],
         [ 33.,  37.,  41.,  45.],
         [ 53.,  57.,  61.,  65.],
         [ 73.,  77.,  81.,  85.]],

        [[ 26.,  34.,  42.,  50.],
         [ 66.,  74.,  82.,  90.],
         [106., 114., 122., 130.],
         [146., 154., 162., 170.]]]])

In [26]:
MaxPool2D(list_b)#Zの出力結果

(array([[[[ 37.,  45.],
          [ 77.,  85.]],
 
         [[ 74.,  90.],
          [154., 170.]]]]), array([[[[0., 0., 0., 0.],
          [0., 1., 0., 1.],
          [0., 0., 0., 0.],
          [0., 1., 0., 1.]],
 
         [[0., 0., 0., 0.],
          [0., 1., 0., 1.],
          [0., 0., 0., 0.],
          [0., 1., 0., 1.]]]]))

In [27]:
def Nh_in(Nh_out,Ph=0,Fh=2,Sh=1):
    Nh_in = (Nh_out -1)*Sh -2*Ph + Fh
    return int(Nh_in)

In [28]:
print(Nh_in(4))

5


In [29]:
def Nw_in(Nw_out,Pw=0,Fw=2,Sw=1):
    Nw_in = (Nw_out -1)*Sw -2*Pw + Fw
    return int(Nw_in)

In [30]:
print(Nw_in(4))

5


In [31]:
#バックプロパゲーション用テストデータ
dA_test1 = np.array([range(32)]).reshape(1,2,4,4)
print(dA_test1)

[[[[ 0  1  2  3]
   [ 4  5  6  7]
   [ 8  9 10 11]
   [12 13 14 15]]

  [[16 17 18 19]
   [20 21 22 23]
   [24 25 26 27]
   [28 29 30 31]]]]


In [32]:
#バックプロパゲーション用テストデータ
dA_test2 = np.array([range(8)]).reshape(1,2,2,2)
print(dA_test2)

[[[[0 1]
   [2 3]]

  [[4 5]
   [6 7]]]]


In [33]:
Z_index_test1 = np.concatenate([Z_index, Z_index], 1)

In [34]:
Z_index_test1.shape

(1, 2, 4, 4)

In [35]:
Z_index_test2 = np.array([0,1,0,1,1,0,0,1,1,0,0,0,1,0,0,1,0,1]).reshape(1,2,3,3)
print(Z_index_test2)

[[[[0 1 0]
   [1 1 0]
   [0 1 1]]

  [[0 0 0]
   [1 0 0]
   [1 0 1]]]]


In [36]:
#バックプロパゲーションのテスト
def backward(dA,Z_index,X,W):#dB,dW
    sample = dA.shape[0]
    chanel = dA.shape[1]
    height = dA.shape[2]
    width = dA.shape[3]  
    
    c_out = W.shape[0]
    c_in = W.shape[1]
    w_height = W.shape[2]
    w_width = W.shape[3]
    
    dW = np.zeros([sample,c_out,c_in,w_height,w_width])

    Z_height = Z_index.shape[2]
    Z_width = Z_index.shape[3]

    h_in = Nh_in(Nh_out=height*2,Ph=0,Fh=2,Sh=1)
    w_in = Nw_in(Nw_out=width*2,Pw=0,Fw=2,Sw=1)

    #Z_index(サンプル,チャネル,高さ,幅）
    
    dB = np.zeros([sample,chanel])
    
    
    dZ_box = np.zeros([sample,c_out,c_in,height,width,Z_height,Z_width])#(1,4,2,2,2,4,4)
#     dZ_box = np.zeros([sample,chanel,height,width,Z_height,Z_width])#(1,2,2,2,4,4)
    dZ = np.zeros([sample,c_out,c_in,h_in,w_in])
#     print(dW.shape)
#     print(dB)
#     print(dW_box.shape)
    #dBの計算
    for q in range(sample):#サンプルを設定
        for i in range(chanel):#チャネルを設定
            dB[q,i] = np.sum(dA[q,i,:,:])

    #dWの計算        
    for r in range(sample):#サンプルを設定
        for s in range(c_out):#出力のチャネルを設定 
            for v in range(c_in):#入力のチャネルを設定
                for t in range(height):#高さを設定
                    for u in range(width):#幅を設定  
                        dW[r,s,v] += dA[r,s,t,u]*X[r,v,t:t + w_height,u:u + w_width]*Z_index[r,s]
#     print(dW)

    #dZの計算
    for h in range(sample):#サンプルを設定
        for j in range(c_out):#出力のチャネルを設定 
            for w in range(c_in):#入力のチャネルを設定
                for k in range(height):#高さを設定
                    for l in range(width):#幅を設定
    #                     print(dA[h,j,k,l])
    #                     print(Z_index[h,j])
    #                     print(dA[h,j,k,l]*Z_index[h,j])
                        dZ_box[h,j,w,k,l] = dA[h,j,k,l]*W[j,k]
#                         dZ_box[h,j,w,k,l] = dA[h,j,k,l]*Z_index[h,j]
    
    for o in range(sample):#サンプルを設定
        for p in range(c_out):#出力チャネルを設定
            for x in range(c_in):#入力チャネルを設定
                for m in range(height):#高さを設定
                    for n in range(width):#幅を設定
                        dZ[o,p,x,m:m+4,n:n+4] += dZ_box[o,p,x,m,n]
    dZ = dZ.sum(axis=1)#出力チャネルでSUM
     
#     print(dW.shape)
#     print(dW[0,4,4])
    return dB, dW, dZ

In [37]:
X = np.random.randint(0, 10, (2,1,5, 5))

In [38]:
W1 = np.array([1,1,1,1,2,2,2,2,1,1,1,1,2,2,2,2,3,3,3,3,2,2,2,2,3,3,3,3,2,2,2,2]).reshape(2,1,4,4)

In [39]:
#バックワードテスト用データdA（サンプル:1、出力チャネル:4、高さ:2、幅:2）
dA_test3 = np.concatenate([dA_test2, dA_test2], 1)
dA_test3.shape

(1, 4, 2, 2)

In [40]:
#バックワードテスト用データZインデック（MAXPOOL時のフィルタ）（サンプル:1、出力チャネル:4、高さ:4、幅:4）
Z_index_test3 = np.concatenate([Z_index_test1, Z_index_test1], 1)
Z_index_test3.shape

(1, 4, 4, 4)

In [41]:
#バックワードテスト用データX（サンプル:1、チャネル:2、高さ:5、幅:5)
X3 = np.random.randint(0, 10, (1,2,5, 5))
X3.shape

(1, 2, 5, 5)

In [42]:
#バックワードテスト用データW（出力チャネル:4、入力チャネル:2、高さ:4、幅:4)
W3 = np.concatenate([W1, W1], 0)
W3 = np.concatenate([W3, W3], 1)
W3.shape

(4, 2, 4, 4)

In [43]:
dB3, dW3, dZ3 = backward(dA_test3,Z_index_test3,X3,W3)

In [44]:
print(dB3.shape)# (サンプル:1、出力チャネル:4)
print(dW3.shape)# (サンプル:1、出力チャネル:4、入力チャネル:2、フィルタ高さ:4、フィルタ幅:4)
print(dZ3.shape)# (サンプル:1、入力チャネル:2、入力特徴量の高さ:5、入力特徴量の幅:5)
# print(dB3)
# print(dW3)
# print(dZ3)

(1, 4)
(1, 4, 2, 4, 4)
(1, 2, 5, 5)


In [45]:
#バックワードテスト用データdA（サンプル:2、出力チャネル:4、高さ:2、幅:2）
dA_test4 = np.concatenate([dA_test3, dA_test3], 0)
dA_test4.shape

(2, 4, 2, 2)

In [46]:
#バックワードテスト用データZインデック（MAXPOOL時のフィルタ）（サンプル:2、出力チャネル:4、高さ:4、幅:4）
Z_index_test4 = np.concatenate([Z_index_test3, Z_index_test3], 0)
Z_index_test4.shape

(2, 4, 4, 4)

In [47]:
#バックワードテスト用データX（サンプル:2、チャネル:2、高さ:5、幅:5)
X4 = np.concatenate([X3, X3], 0)
X4.shape

(2, 2, 5, 5)

In [48]:
#バックワードテスト用データW（出力チャネル:4、入力チャネル:2、高さ:4、幅:4)
W4 = W3
W4.shape

(4, 2, 4, 4)

In [49]:
dB4, dW4, dZ4 = backward(dA_test4,Z_index_test4,X4,W4)

In [50]:
print(dB4.shape)# (サンプル:2、出力チャネル:4)
print(dW4.shape)# (サンプル:2、出力チャネル:4、入力チャネル:2、フィルタ高さ:4、フィルタ幅:4)
print(dZ4.shape)# (サンプル:2、入力チャネル:2、入力特徴量の高さ:5、入力特徴量の幅:5)
# print(dB4)
# print(dW4)
# print(dZ4)

(2, 4)
(2, 4, 2, 4, 4)
(2, 2, 5, 5)


In [51]:
#conv2d用のイニシャライザー
class ConV2d_Initializer:
    """
    ガウス分布によるシンプルな初期化
    Parameters
    ----------
    sigma : float
      ガウス分布の標準偏差
    """
    def __init__(self, sigma=1, in_cha=2, out_cha=4, F_size=4):
        self.sigma = sigma
        self.in_cha = in_cha
        self.out_cha = out_cha
        self.F_size = F_size
        
    def W(self):
        W = np.random.randn(self.out_cha, self.in_cha, self.F_size, self.F_size)
#         print(W.shape)
        pass
        return W
    def B(self):
        B = self.sigma * np.random.rand(self.out_cha)
#         print(B.shape)
        pass
        return B

In [52]:
cv2 = ConV2d_Initializer()
print(cv2.W().shape)
print(cv2.B().shape)

(4, 2, 4, 4)
(4,)


In [53]:
#テスト用の最適化関数
class SGD:
    """
    確率的勾配降下法
    Parameters
    ----------
    lr : 学習率
    """
    def __init__(self, lr=0.001):
        self.lr = lr
    def update(self, layer):
        layer.W = layer.W - self.lr*layer.dW
        layer.B = layer.B - self.lr*layer.dB.mean(axis=0)
        return layer

In [54]:
#Conv2d のクラス化

class Conv2d:

    def __init__(self, Nh_in, Nw_in, initializer=ConV2d_Initializer(), optimizer=SGD,Ph=0,Fh=2,Sh=1,Pw=0,Fw=2,Sw=1):
#     def __init__(self, N_in=2, B, initializer=ones_Initializer, optimizer=SGD, F=3,S=1,P=0):
        self.initializer = initializer
        self.optimizer = optimizer
        
        ini = self.initializer   
        opt = self.optimizer
        
        #初期値を決める
        # WとBを決定する
        self.W = ini.W()
        self.B = ini.B() 
        
        self.h_in = Nh_in
#         print(self.h_in)
        self.Ph = Ph
        self.Fh = Fh
        self.Sh = Sh
        self.w_in = Nw_in
        self.Pw = Pw
        self.Fw = Fw
        self.Sw = Sw
        self.Nh_out()
#         print(self.h_out)
        self.Nw_out()
#         print(self.w_out)
 
    def Nh_out(self,Ph=0,Fh=4,Sh=1):#Fh フィルターサイズて入力
        self.h_out = int((self.h_in + 2*Ph - Fh)/Sh + 1)

    def Nw_out(self,Pw=0,Fw=4,Sw=1):#Fw フィルターサイズて入力
        self.w_out = int((self.w_in + 2*Pw - Fw)/Sw + 1)

    def forward(self,X): # W,Bのデータをself.で引き出せるようにする
        self.X = X
        sample = X.shape[0]
        chanel = X.shape[1]
        height = X.shape[2]
        width = X.shape[3]

        c_out = self.W.shape[0]
#         print(c_out)
        c_in = self.W.shape[1]
#         print(c_in)
        w_height = self.W.shape[2]
        w_width = self.W.shape[3]

#         h_out = Nh_out(Nh_in=height,Ph=0,Fh=2,Sh=1)
#         w_out = Nw_out(Nw_in=width,Pw=0,Fw=2,Sw=1)
        A0_list = np.zeros([sample,c_out,c_in,self.h_out,self.w_out])
    #     A_list = []
    #     print(h_out)
    #     print(w_out)
#         print(A0_list.shape)

        # X*Wの計算
        for h in range(sample):#サンプルを設定
            for k in range(chanel):#チャネルを設定
                for i in range(0, self.h_out, 1):#高さを設定
                    for j in range(0, self.w_out, 1):#幅を設定
                        A0 = X[h,k,i:i + w_height,j:j + w_width]#Xの中の要素を抽出
    #                     print(A0)
                        for l in range(c_out):#フィルターの出力チャネルを設定
                            for m in range(c_in):#フィルターの入力チャネルを設定
                                A0_list[h,l,m,i,j] = (A0*self.W[l,m]).sum()

        A0 = A0_list.sum(axis=2)#入力チャネル方向にSUM
#         print(A0.shape)
        # +Bの計算
        for n in range(sample):#サンプルを設定
            for o in range(c_out):#フィルターの出力チャネルを設定
                A0[n,o,:,:] += self.B[o]

        self.A = A0
        return A0#([サンプル数,チャネル,高さ,幅])
      
    def backward(self,dA,Z_index): # Z_index,X,Wのデータをself.で引き出せるようにする
        sample = dA.shape[0]
        chanel = dA.shape[1]
        height = dA.shape[2]
        width = dA.shape[3]  

        c_out = self.W.shape[0]
#         print(c_out)
        c_in = self.W.shape[1]
#         print(c_in)
        w_height = self.W.shape[2]
        w_width = self.W.shape[3]

        dW = np.zeros([sample,c_out,c_in,w_height,w_width])

        Z_height = Z_index.shape[2]
        Z_width = Z_index.shape[3]

#         h_in = Nh_in(Nh_out=height*2,Ph=0,Fh=2,Sh=1)
#         w_in = Nw_in(Nw_out=width*2,Pw=0,Fw=2,Sw=1)

        #Z_index(サンプル,チャネル,高さ,幅）

        dB = np.zeros([sample,chanel])


        dZ_box = np.zeros([sample,c_out,c_in,height,width,Z_height,Z_width])#(1,4,2,2,2,4,4)
    #     dZ_box = np.zeros([sample,chanel,height,width,Z_height,Z_width])#(1,2,2,2,4,4)
        dZ = np.zeros([sample,c_out,c_in,self.h_in,self.w_in])
    #     print(dW.shape)
    #     print(dB)
    #     print(dW_box.shape)
        #dBの計算
        for q in range(sample):#サンプルを設定
            for i in range(chanel):#チャネルを設定
                dB[q,i] = np.sum(dA[q,i,:,:])
        self.dB = dB

        #dWの計算        
        for r in range(sample):#サンプルを設定
            for s in range(c_out):#出力のチャネルを設定 
                for v in range(c_in):#入力のチャネルを設定
                    for t in range(height):#高さを設定
                        for u in range(width):#幅を設定  
                            dW[r,s,v] += dA[r,s,t,u]*self.X[r,v,t:t + w_height,u:u + w_width]*Z_index[r,s]
        self.dW = dW

        #dZの計算
        for h in range(sample):#サンプルを設定
            for j in range(c_out):#出力のチャネルを設定 
                for w in range(c_in):#入力のチャネルを設定
                    for k in range(height):#高さを設定
                        for l in range(width):#幅を設定
        #                     print(dA[h,j,k,l])
        #                     print(Z_index[h,j])
        #                     print(dA[h,j,k,l]*Z_index[h,j])
                            dZ_box[h,j,w,k,l] = dA[h,j,k,l]*self.W[j,k]
    #                         dZ_box[h,j,w,k,l] = dA[h,j,k,l]*Z_index[h,j]

        for o in range(sample):#サンプルを設定
            for p in range(c_out):#出力チャネルを設定
                for x in range(c_in):#入力チャネルを設定
                    for m in range(height):#高さを設定
                        for n in range(width):#幅を設定
                            dZ[o,p,x,m:m+4,n:n+4] += dZ_box[o,p,x,m,n]
        dZ = dZ.sum(axis=1)#出力チャネルでSUM

    #     print(dW.shape)
    #     print(dW[0,4,4])
    
#         self.optimizer.update(self)
        
        return dZ


In [55]:
cv2d = Conv2d(5,5)

In [56]:
X4.shape

(2, 2, 5, 5)

In [57]:
(cv2d.forward(X4)).shape# dAのシェイプは（サンプル数：2,チャネル数：4,高さ：2,幅：2）　OK

(2, 4, 2, 2)

In [58]:
(cv2d.backward(dA_test4,Z_index_test4)).shape # dZのシェイプは（サンプル数：2,チャネル数：2,高さ：5,幅：5）　OK

(2, 2, 5, 5)

# 【問題5】平滑化

In [59]:
class Flatten:
    def __init__(self):
        
        pass
    def forward(self,X):
        self.X = X
        self.figure = X.shape
        
        X_F = X.reshape(X.shape[0],-1)
        self.X_F = X_F
        return X_F
    
    def backward(self,dA_F):
        self.dA_F = dA_F
        dA = dA_F.reshape(self.figure)
#         print(dA)
        return dA

In [60]:
ft = Flatten()

In [61]:
X_F = ft.forward(X4)#X4(サンプル数：2,2,5,5)の平滑化　OK
X_F.shape

(2, 50)

In [62]:
dA = ft.backward(X_F)#X4(2,2,5,5)の平滑化もどし　OK
dA.shape

(2, 2, 5, 5)

# 【問題6】学習と推定

In [63]:
class SimpleInitializer:
    """
    ガウス分布によるシンプルな初期化
    Parameters
    ----------
    sigma : float
      ガウス分布の標準偏差
    """
    def __init__(self, sigma=0.005):
        self.sigma = sigma
        
    def W(self, n_nodes1, n_nodes2):
        W = self.sigma * np.random.randn(n_nodes1, n_nodes2)
        pass
        return W
    
    def B(self, n_nodes2):
        B = self.sigma * np.random.rand(n_nodes2)
        pass
        return B

In [64]:
class FC:
    """
    ノード数n_nodes1からn_nodes2への全結合層
    Parameters
    ----------
    n_nodes1 : int
      前の層のノード数
    n_nodes2 : int
      後の層のノード数
    initializer : 初期化方法のインスタンス
    optimizer : 最適化手法のインスタンス
    """
    
    def __init__(self, n_nodes1, n_nodes2, initializer=SimpleInitializer, optimizer=SGD):
        self.initializer = SimpleInitializer
        self.optimizer = optimizer
        self.n_nodes1 = n_nodes1
        self.n_nodes2 = n_nodes2
        
        ini = self.initializer()
        #初期値を決める
        # WとBを決定する
        self.W = ini.W(self.n_nodes1,self.n_nodes2)
        self.B = ini.B(self.n_nodes2)        
        
        opt = self.optimizer
        #最適化処理をインスタンス化
        
        pass
    def forward(self, Z):
        self.Z = Z
        A = self.Z@self.W + self.B
        self.A = A
        
        pass
        return A
    def backward(self, dA):
        self.dA = dA
        self.dB = self.dA.mean(axis=0)
        self.dW = self.Z.T@self.dA
        dZ = self.dA@self.W.T
        
        pass
        # 更新
        self.optimizer.update(self)
        return dZ

In [65]:
# タンジェントハイポ
class tanh:
    def __init__(self):
        
        pass
    def forward(self, A):
        self.A = A
        Z = np.tanh(A)
        self.Z = Z       
        return Z        
    def backward(self, dZ):
        dA = dZ*(1-np.tanh(self.A)**2)
    
        return dA

In [66]:
#ソフトマックス
class softmax:
    def __init__(self):
        
        pass
    def forward(self, A):
        Z = (np.exp(A).T/np.exp(A).sum(axis=1)).T
         
        return Z
    def backward(self, Z, Y):
        dA = Z - Y
        
        return dA

In [67]:
# ミニバッチ
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 20, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self._X = X[shuffle_index]
        self._y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self._X[p0:p1], self._y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self._X[p0:p1], self._y[p0:p1]

In [69]:
#conv2d用のイニシャライザー
class ConV2d_Initializer_pred_test:
    """
    ガウス分布によるシンプルな初期化
    Parameters
    ----------
    sigma : float
      ガウス分布の標準偏差
    """
    def __init__(self, sigma=1, in_cha=1, out_cha=2, F_size=2):
        self.sigma = sigma
        self.in_cha = in_cha
        self.out_cha = out_cha
        self.F_size = F_size
        
    def W(self):
        W = np.random.randn(self.out_cha, self.in_cha, self.F_size, self.F_size)
#         print(W.shape)
        pass
        return W
    def B(self):
        B = self.sigma * np.random.rand(self.out_cha)
#         print(B.shape)
        pass
        return B

In [70]:
#Conv2d のクラス化

class Conv2d_test_pred:

    def __init__(self, Nh_in, Nw_in, initializer=ConV2d_Initializer_pred_test(), optimizer=SGD,Ph=0,Fh=2,Sh=1,Pw=0,Fw=2,Sw=1):
#     def __init__(self, N_in=2, B, initializer=ones_Initializer, optimizer=SGD, F=3,S=1,P=0):
        self.initializer = initializer
        self.optimizer = optimizer
        
        ini = self.initializer   
        opt = self.optimizer
        
        #初期値を決める
        # WとBを決定する
        self.W = ini.W()
        self.B = ini.B() 
        
        self.h_in = Nh_in
#         print(self.h_in)
        self.Ph = Ph
        self.Fh = Fh
        self.Sh = Sh
        self.w_in = Nw_in
        self.Pw = Pw
        self.Fw = Fw
        self.Sw = Sw
        self.Nh_out()
#         print(self.h_out)
        self.Nw_out()
#         print(self.w_out)
 
    def Nh_out(self,Ph=0,Fh=2,Sh=1):#Fh フィルターサイズて入力
        self.h_out = int((self.h_in + 2*Ph - Fh)/Sh + 1)

    def Nw_out(self,Pw=0,Fw=2,Sw=1):#Fw フィルターサイズて入力
        self.w_out = int((self.w_in + 2*Pw - Fw)/Sw + 1)

    def forward(self,X): # W,Bのデータをself.で引き出せるようにする
        self.X = X
        sample = X.shape[0]
        chanel = X.shape[1]
        height = X.shape[2]
        width = X.shape[3]

        c_out = self.W.shape[0]
#         print(c_out)
        c_in = self.W.shape[1]
#         print(c_in)
        w_height = self.W.shape[2]
        w_width = self.W.shape[3]

#         h_out = Nh_out(Nh_in=height,Ph=0,Fh=2,Sh=1)
#         w_out = Nw_out(Nw_in=width,Pw=0,Fw=2,Sw=1)
        A0_list = np.zeros([sample,c_out,c_in,self.h_out,self.w_out])
    #     A_list = []
    #     print(h_out)
    #     print(w_out)
#         print(A0_list.shape)

        # X*Wの計算
        for h in range(sample):#サンプルを設定
            for k in range(chanel):#チャネルを設定
                for i in range(0, self.h_out, 1):#高さを設定
                    for j in range(0, self.w_out, 1):#幅を設定
                        A0 = X[h,k,i:i + w_height,j:j + w_width]#Xの中の要素を抽出
    #                     print(A0)
                        for l in range(c_out):#フィルターの出力チャネルを設定
                            for m in range(c_in):#フィルターの入力チャネルを設定
                                A0_list[h,l,m,i,j] = (A0*self.W[l,m]).sum()

        A0 = A0_list.sum(axis=2)#入力チャネル方向にSUM
#         print(A0.shape)
        # +Bの計算
        for n in range(sample):#サンプルを設定
            for o in range(c_out):#フィルターの出力チャネルを設定
                A0[n,o,:,:] += self.B[o]

        self.A = A0
        return A0#([サンプル数,チャネル,高さ,幅])
      
    def backward(self,dA,M_index): # Z_index,X,Wのデータをself.で引き出せるようにする
        sample = dA.shape[0]
        chanel = dA.shape[1]
        height = dA.shape[2]
        width = dA.shape[3]  

        c_out = self.W.shape[0]
#         print(c_out)
        c_in = self.W.shape[1]
#         print(c_in)
        w_height = self.W.shape[2]
        w_width = self.W.shape[3]

        dW = np.zeros([sample,c_out,c_in,w_height,w_width])

        Z_height = Z_index.shape[2]
        Z_width = Z_index.shape[3]

#         h_in = Nh_in(Nh_out=height*2,Ph=0,Fh=2,Sh=1)
#         w_in = Nw_in(Nw_out=width*2,Pw=0,Fw=2,Sw=1)

        #Z_index(サンプル,チャネル,高さ,幅）

        dB = np.zeros([sample,chanel])


        dZ_box = np.zeros([sample,c_out,c_in,height,width,Z_height,Z_width])#(1,4,2,2,2,4,4)
    #     dZ_box = np.zeros([sample,chanel,height,width,Z_height,Z_width])#(1,2,2,2,4,4)
        dZ = np.zeros([sample,c_out,c_in,self.h_in,self.w_in])
    #     print(dW.shape)
    #     print(dB)
    #     print(dW_box.shape)
        #dBの計算
        for q in range(sample):#サンプルを設定
            for i in range(chanel):#チャネルを設定
                dB[q,i] = np.sum(dA[q,i,:,:])
        self.dB = dB

        #dWの計算        
        for r in range(sample):#サンプルを設定
            for s in range(c_out):#出力のチャネルを設定 
                for v in range(c_in):#入力のチャネルを設定
                    for t in range(height):#高さを設定
                        for u in range(width):#幅を設定  
                            dW[r,s,v] += dA[r,s,t,u]*self.X[r,v,t:t + w_height,u:u + w_width]*Z_index[r,s]
        self.dW = dW

        #dZの計算
        for h in range(sample):#サンプルを設定
            for j in range(c_out):#出力のチャネルを設定 
                for w in range(c_in):#入力のチャネルを設定
                    for k in range(height):#高さを設定
                        for l in range(width):#幅を設定
        #                     print(dA[h,j,k,l])
        #                     print(Z_index[h,j])
        #                     print(dA[h,j,k,l]*Z_index[h,j])
                            dZ_box[h,j,w,k,l] = dA[h,j,k,l]*self.W[j,k]
    #                         dZ_box[h,j,w,k,l] = dA[h,j,k,l]*Z_index[h,j]

        for o in range(sample):#サンプルを設定
            for p in range(c_out):#出力チャネルを設定
                for x in range(c_in):#入力チャネルを設定
                    for m in range(height):#高さを設定
                        for n in range(width):#幅を設定
                            dZ[o,p,x,m:m+4,n:n+4] += dZ_box[o,p,x,m,n]
        dZ = dZ.sum(axis=1)#出力チャネルでSUM

    #     print(dW.shape)
    #     print(dW[0,4,4])
    
#         self.optimizer.update(self)
        
        return dZ

In [71]:
#入力はNHWC(n_samples, height, width, n_channels)
class MaxPool2D:
    def M(self,A):
        sample = A.shape[0]
        chanel = A.shape[1]
        height = A.shape[2]
        width = A.shape[3]
        Z_index = np.zeros([sample,chanel,height,width])
        Z = np.zeros([sample,chanel,int(height/2),int(width/2)])
    #     print(Z.shape)

        for h in range(sample):#サンプルを設定
            for k in range(chanel):#チャネルを設定
                for i in range(0, int(height/2), 1):#高さを設定
                    for j in range(0, int(width/2), 1):#幅を設定
                            Z0 = A[h,k,i*2:i*2 + 2,j*2:j*2 + 2]
                            Z[h,k,i,j] = np.nanmax(Z0)
                            a = Z0/np.nanmax(Z0)#最大値が１の行列
                            Z_index[h,k,i*2:i*2 + 2,j*2:j*2 + 2] += (np.where(a==1,1,0))


        self.Z = Z
        self.Z_index = Z_index
        return Z

In [76]:
class ScratchDeepNeuralNetrowkClassifier():

    def __init__(self,num_iter = 1,lr = 0.01, verbose = True):
        self.verbose = verbose
        self.num_iter = num_iter
        self.lr = lr
        self.verbose = verbose
#         self.n_features = 784#
        self.n_nodes1 = 338
    #
#         self.n_nodes2 = 200#
        self.n_output = 10
        self.alpha = 0.001
        self.rec_loss = [] 
        self.rec_val_loss = []         
        
        pass
    def fit(self, X, y, X_val=None, y_val=None):
        #最適化手法のインスタンス化
        optimizer = SGD(self.lr)
        initializer1 = ConV2d_Initializer_pred_test(2,2)
        initializer2 = SimpleInitializer()
        #層の数、各層の特徴数（インプット＆アウトプット）、初期値設定、最適化手法、活性化関数の種類を設定
        self.MP = MaxPool2D()
        self.Cv2d1 = Conv2d_test_pred(28, 28)#, initializer1, optimizer)
        self.activation1 = Flatten()
        self.FC1 = FC(self.n_nodes1, self.n_output, initializer2, optimizer)
        self.activation2 = softmax()

        #ミニバッチ２０で学習
        self.X = X
        self.y = y
        self.X_val = X_val
        self.y_val = y_val
        get_mini_batch = GetMiniBatch(self.X, self.y, batch_size=20)
        self.z_all = np.empty((0, 10),dtype=np.float)
        for i in range(self.num_iter):
            for mini_X_train, mini_y_train in get_mini_batch:
        # フォワードプロパゲーション
                A1 = self.Cv2d1.forward(mini_X_train)
                M1 = self.MP.M(A1)
                Z1 = self.activation1.forward(M1)
#                 print(Z1.shape)
                A2 = self.FC1.forward(Z1)
                Z2 = self.activation2.forward(A2)
#                 A3 = self.FC3.forward(Z2)
#                 Z3 = self.activation3.forward(A3)
                self.Z2 = Z2
        # バックプロパゲーション
                dA2 = self.activation2.backward(Z2, mini_y_train) # 交差エントロピー誤差とソフトマックスを合わせている
                dZ1 = self.FC1.backward(dA2)
#                 dA2 = self.activation2.backward(dZ2)
#                 dZ1 = self.FC2.backward(dA2)
                dA1 = self.activation1.backward(dZ1)
                index = self.MP.Z_index
#                 print(index.shape)
#                 print(dA1.shape)
#                 dZ0 = self.Cv2d1.backward(dA1,index) # dZ0は使用しない
#             self.loss_entropy(mini_y_train)
            #フィット後のB、Wを抜き出す
            self.B1 = self.Cv2d1.B
            self.B2 = self.FC1.B
#             self.B3 = self.FC3.B
            
            self.W1 = self.Cv2d1.W
            self.W2 = self.FC1.W
#             self.W3 = self.FC3.W
#             self.val_loss_entropy()
                
        if self.verbose:
            #verboseをTrueにした際は学習過程などを出力する
            print()
        pass

#     def loss_entropy(self,y_train_batch):
#         self.loss = -1/self.Z3.shape[0]*(y_train_batch*(np.log(self.Z3))).sum()  
#         self.rec_loss.append(self.loss)
        
#     def val_loss_entropy(self):
#         self.a1 = self.X_val@self.W1 + self.B1
#         self.z1 = np.tanh(self.a1)
#         self.a2 = self.z1@self.W2 + self.B2
#         self.z2 = np.tanh(self.a2)
#         self.a3 = self.z2@self.W3 + self.B3
#         self.z3 = self.activation3.forward(self.a3)        
#         self.val_loss = -1/self.z3.shape[0]*(self.y_val*(np.log(self.z3))).sum()  
#         self.rec_val_loss.append(self.val_loss)
        
    def predict(self, X_test):
        z_all = np.empty((0, 10),dtype=np.float)
#         print(z_all.shape)
        self.a1 = self.Cv2d1.forward(X_test)
        self.z1 = self.activation1.forward(self.a1)
        self.a2 = self.z1@self.W2 + self.B2
#         self.z2 = np.tanh(self.a2)
#         self.a3 = self.z2@self.W3 + self.B3
        self.z2 = self.activation2.forward(self.a2)
        z_all = np.concatenate([z_all,self.z2])
        z_all_index = np.argmax(z_all, axis=1)
        
        pass
        return z_all, z_all_index

In [77]:
clf = ScratchDeepNeuralNetrowkClassifier()
clf.fit(X_train_0, y_train_0, X_val_0, y_val_0)

In [84]:
print(X_train_0.shape)

(600, 1, 28, 28)


# 【問題9】出力サイズとパラメータ数の計算

CNNモデルを構築する際には、全結合層に入力する段階で特徴量がいくつになっているかを事前に計算する必要があります。<br>
また、巨大なモデルを扱うようになると、メモリや計算速度の関係でパラメータ数の計算は必須になってきます。<br>
フレームワークでは各層のパラメータ数を表示させることが可能ですが、意味を理解していなくては適切な調整が行えません。<br>
以下の3つの畳み込み層の出力サイズとパラメータ数を計算してください。パラメータ数についてはバイアス項も考えてください。<br>

1.
入力サイズ : 144×144, 3チャンネル<br>
フィルタサイズ : 3×3, 6チャンネル<br>
ストライド : 1<br>
パディング : なし<br>


A.1<br>
出力サイズ：１４２＊１４２<br>
出力チャネル：６<br>
パラメータ数：３＊３＊６　＋　６<br>
            ＝60<br>

2.
入力サイズ : 60×60, 24チャンネル<br>
フィルタサイズ : 3×3, 48チャンネル<br>
ストライド　: 1<br>
パディング : なし<br>

A.2<br>
出力サイズ：58＊58<br>
出力チャネル：48<br>
パラメータ数：3＊3＊48　＋　48<br>
            ＝480<br>

3.
入力サイズ : 20×20, 10チャンネル<br>
フィルタサイズ: 3×3, 20チャンネル<br>
ストライド : 2<br>
パディング : なし<br>

＊最後の例は丁度良く畳み込みをすることができない場合です。<br>
フレームワークでは余ったピクセルを見ないという処理が行われることがあるので、その場合を考えて計算してください。<br>
端が欠けてしまうので、こういった設定は好ましくないという例です。<br>

A.3<br>
出力サイズ：9＊9<br>
出力チャネル：20<br>
パラメータ数：9＊9＊20　＋　20<br>
            ＝200<br>